# 空间编码 (Spatial Encoding) 可视化实验




In [5]:
import numpy as np
import matplotlib.pyplot as plt

def get_spatial_encoding(coord, D=128, base=10000):
    """
    实现经典的正弦/余弦空间编码公式。
    
    参数:
    coord: 标量值 (如纬度、经度或时间 t)
    D:    编码的维度 (通常为偶数)
    base:  波长几何级数的底数 (Transformer论文默认为10000)
    
    返回:
    pe:   形状为 (D,) 的 numpy 数组
    """
    pe = np.zeros(D)
    for k in range(D // 2):
        # 1. 计算不同频段的分母：base^(2k/D)
        #    k=0 时分母为 1 (高频)
        #    k 大时分母很大 (低频，波长长)
        div_term = np.power(base, (2 * k) / D)
        
        # 2. 偶数位用 sin，奇数位用 cos
        pe[2 * k] = np.sin(coord / div_term)
        pe[2 * k + 1] = np.cos(coord / div_term)
    return pe

## 1. 数学原理

对经度进行以下操作，获得编码向量

$$
\begin{aligned}
PE(l_{lat}, 2k) &= \sin(\frac{l_{lat}}{10000^{2k/D}}) \\
PE(l_{lat}, 2k + 1) &= \cos(\frac{l_{lat}}{10000^{2k/D}})
\end{aligned}
\quad k = 0, 1, ..., \frac{D}{2} - 1
$$

同理，对纬度也进行如下操作

$$
\begin{aligned}
PE(l_{lon}, 2k) &= \sin(\frac{l_{lon}}{10000^{2k/D}}) \\
PE(l_{lon}, 2k + 1) &= \cos(\frac{l_{lon}}{10000^{2k/D}})
\end{aligned}
\quad k = 0, 1, ..., \frac{D}{2} - 1
$$

> Q1:为什么要对位置信息进行编码操作？

> A1:为了解决非线性，非周期，破坏语义信息三个问题

- 非线性：经纬度和地理位置的对应关系并非线性表示
- 非周期：空间邻近性的断裂，比如179.9和0
- 破坏语义信息：都是首都，但是简单的经纬度并不能体现这一点（查的距离很远，但是特点可能相似，这时候需要我们用低频的角度去看）

> Q2:为什么要对奇偶不同的k进行正余弦变换

> A2:因为$PE_k(x) = \sin(\omega_k x)$是有歧义的，因为$\sin(x) = \sin(\pi - x)$所以不同位置可能编码一样，比如函数值1/2对应了30度和150度，但是分为$(\sin(\omega x), \cos(\omega x))$，则可以表示单位圆上的一个点

>Q3:这个k是什么？

>A3:表示频率的敏感度，高频的话就对一些细微的差距很敏感，可以用于识别小范围的变化（比如下个十字路口和这个十字路口的区别）。低频则是对更宏观的变化的识别（比如跨城市）


---
## 2. 数据模拟
我们通过交互式实验模拟两个坐标点：
*   **A**: 基准点 (固定为纽约纬度)
*   **B**: 距离 A 一定距离的点 ，距离可以通过滑块控制，用来模拟不同距离的点

### 可交互参数

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# =======================================================
# 核心实验：理解 "Base (底数)" 对频率分布的影响
# =======================================================
# 文档核心思想: 
# 1. PE(p, 2i) = sin(p / base^(2i/D))
# 2. Base 控制了波长范围的跨度 (Geometric Sequence)。
#    - Base 越大：频率衰减越快，低频部分波长极大，能捕捉极长距离的特征。
#    - Base 越小：频率衰减越慢，高低频差异小，更关注局部细节。

def interact_base_experiment(distance_km, base_val):
    # 模拟数据：
    # 1. 经纬度变化量 delta (1度 ≈ 111km)
    delta_deg = distance_km / 111.0 
    
    # 2. 两个位置点
    raw_loc_A = 40.7128 # 纽约纬度
    raw_loc_B = raw_loc_A + delta_deg
    
    # 3. 实验设置
    fixed_scale = 1000.0 
    
    # 使用可变 Base 进行编码
    v_A = get_spatial_encoding(raw_loc_A * fixed_scale, D=128, base=base_val)
    v_B = get_spatial_encoding(raw_loc_B * fixed_scale, D=128, base=base_val)
    
    # 计算余弦相似度
    sim = np.dot(v_A, v_B) / (np.linalg.norm(v_A) * np.linalg.norm(v_B))
    
    # --- 绘图 ---
    plt.figure(figsize=(15, 6))
    
    # [左图] 前 20 维 (相对高频)
    plt.subplot(1, 2, 1)
    plt.plot(v_A[:20], label='Loc A', marker='o', alpha=0.7)
    plt.plot(v_B[:20], label='Loc B', marker='x', linestyle='--', alpha=0.7)
    plt.title(f"Higher Frequency (First 20)\nDistance={distance_km}km | Base={base_val}")
    plt.xlabel("Encoding Index (0-19)")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # [右图] 后 40 维 (相对低频)
    plt.subplot(1, 2, 2)
    # 绘制最后 40 维
    indices_low = np.arange(128-40, 128) 
    plt.plot(indices_low, v_A[-40:], label='Loc A', marker='o', alpha=0.7)
    plt.plot(indices_low, v_B[-40:], label='Loc B', marker='x', linestyle='--', alpha=0.7)
    plt.title(f"Lower Frequency (Last 40)\nSim={sim:.4f}")
    plt.xlabel("Encoding Index (88-127)")
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
# --- 控件设置 ---
style = {'description_width': 'initial'}

# 1. 真实物理距离
dist_slider = widgets.FloatLogSlider(value=100.0, base=10, min=-1, max=4, step=0.1, description='距离(km)', style=style, layout=widgets.Layout(width='100%'))

# 2. Base 底数
# 范围：从 10^0 (1) 到 10^6 (1000000)
base_slider = widgets.FloatLogSlider(value=10000.0, base=10, min=0, max=6, step=0.1, description='Base底数', style=style, layout=widgets.Layout(width='100%'))

print("调节 Base 观察对频率分布的影响：")
widgets.interact(interact_base_experiment, 
                 distance_km=dist_slider, 
                 base_val=base_slider)

调节 Base 观察对频率分布的影响：


interactive(children=(FloatLogSlider(value=100.0, description='距离(km)', layout=Layout(width='100%'), min=-1.0,…

<function __main__.interact_base_experiment(distance_km, base_val)>